In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 3 # piminus, electron

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-50M_piminus_embed_single_new_04-10_v0.root","my-50M_electron_embed_single_new_04-10_v0.root","my-100M_piminus_single_hagedorn_04-5_v0.root"\
            ,"my-100M_piminus_embed_single_new_04-10_v0.root","my-50M_ccbar_pythia_v0.root"]
file_name=file_names[iOption0]
outfilenames= ['pionminus','electron','pionminus1','pionminus2','eccbar']#output
filename=outfilenames[iOption0]
part_ids=[9,3,9,9,3]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]
N_start, N_end = 0e6, 100e6

In [4]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [6]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [7]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetN0()>=0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
              round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
              myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
              round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
              round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

99836813
Nevent: 42, centrality: 58.0, x: 0.33152198791503906, Ntracks: 1
pt:  0.574 , e/p:  0.274 , n0:  1 , disp:  7.04 , chi2/npe0:  857061.61 , prob:  0.87 , id:  -pion
0.5761852860450745
Nevent: 130, centrality: 38.0, x: 0.33743399381637573, Ntracks: 1
pt:  2.844 , e/p:  0.086 , n0:  1 , disp:  7.15 , chi2/npe0:  268473.13 , prob:  0.26 , id:  -pion
2.7054426670074463
Nevent: 252, centrality: 38.0, x: 0.346792995929718, Ntracks: 1
pt:  0.604 , e/p:  0.314 , n0:  0 , disp:  357.21 , chi2/npe0:  521979.45 , prob:  0.922 , id:  -pion
0.6040382385253906
Nevent: 504, centrality: 20.0, x: 0.3515709936618805, Ntracks: 1
pt:  2.041 , e/p:  0.154 , n0:  1 , disp:  9.21 , chi2/npe0:  69.1 , prob:  0.942 , id:  -pion
2.0386810302734375
Nevent: 616, centrality: 2.0, x: 0.33170801401138306, Ntracks: 1
pt:  0.397 , e/p:  0.184 , n0:  0 , disp:  338.21 , chi2/npe0:  0.0 , prob:  0.522 , id:  -pion
0.5057113766670227
Nevent: 805, centrality: 7.0, x: 0.3432289958000183, Ntracks: 1
pt:  0.45 , e/p:

In [8]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0

for ievent in range(int(N_start), min(int(N_end),N_real_ev)):
    if (ievent - N_start) % 1e5 == 0: print("N of events:", ievent - N_start)
    if good_event_counter>100000000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and myevent.GetEntry(ipart).GetN0()>=0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and myevent.GetEntry(ipart).GetN0()>=0:
            data_array.append([int(myevent.GetCentrality()),myevent.GetEntry(ipart).GetZDC(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),min(myevent.GetEntry(ipart).GetChi2(), myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-6)),\
                            myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetEmcdphi(),myevent.GetEntry(ipart).GetEmcdz()])


data = pd.DataFrame(data_array, columns=('centrality','zdc','pt', 'e/p', 'n0', 'disp', 'chi2/npe0', 'npe0', 'prob','emcdphi','emcdz'))

99836813
N of events: 0.0
N of events: 100000.0
N of events: 200000.0
N of events: 300000.0
N of events: 400000.0
N of events: 500000.0
N of events: 600000.0
N of events: 700000.0
N of events: 800000.0
N of events: 900000.0
N of events: 1000000.0
N of events: 1100000.0
N of events: 1200000.0
N of events: 1300000.0
N of events: 1400000.0
N of events: 1500000.0
N of events: 1600000.0
N of events: 1700000.0
N of events: 1800000.0
N of events: 1900000.0
N of events: 2000000.0
N of events: 2100000.0
N of events: 2200000.0
N of events: 2300000.0
N of events: 2400000.0
N of events: 2500000.0
N of events: 2600000.0
N of events: 2700000.0
N of events: 2800000.0
N of events: 2900000.0
N of events: 3000000.0
N of events: 3100000.0
N of events: 3200000.0
N of events: 3300000.0
N of events: 3400000.0
N of events: 3500000.0
N of events: 3600000.0
N of events: 3700000.0
N of events: 3800000.0
N of events: 3900000.0
N of events: 4000000.0
N of events: 4100000.0
N of events: 4200000.0
N of events: 4300

In [9]:
data.head()

,centrality,zdc,pt,e/p,n0,disp,chi2/npe0,npe0,prob,emcdphi,emcdz
0,58,-51.218750,0.574043,0.273976,1,7.036806,85.706161,0,0.869948,0.015658,-4.884285
1,38,19.765625,2.844389,0.085505,1,7.150626,26.847313,0,0.259610,-0.001379,-0.833191
2,38,46.125000,0.603700,0.314283,0,357.211182,52.197945,0,0.921807,-0.073750,26.835541
3,20,-20.281250,2.041460,0.153698,1,9.214996,69.110466,1,0.942313,-0.011559,-1.957794
4,2,-52.968750,0.397347,0.184314,0,338.210846,0.000000,0,0.521526,0.035362,0.040855


In [10]:
print(data.size, data.shape, good_event_counter)

11769747 (1069977, 11) 1063020


In [11]:
filepath = '/home/yoren/bnl/PHENIX/ee/ML/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [12]:
data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)